In [1]:
#Import Dependencies
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
##Source = https://aca5.accela.com/bcc/customization/bcc/cap/licenseSearch.aspx

In [3]:
#California_Cannabis_Distributer_Data
california_data = "../ETL_project/california_data.csv"
california_data_df = pd.read_csv(california_data, encoding="utf-8")
california_data_df.head()

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,"PDC, LLC : Email- license@pmcoc.com : Phone- 7...",Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,WE CARE CAT CITY INC. : GUIDE : Email- George...,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,GREENFIELD HERBS : Greenfield Herbs : Email- ...,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,"BEAVER FREIGHT, INC. : Email- mcastaneda85@gma...",Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,"SESPE CREEK COLLECTIVE, INC. : Email- info@ses...",Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH


In [4]:
#Individual column names in california_data_df
list(california_data_df)

['License Number',
 'License Type',
 'Business Owner',
 'Business Contact Information',
 'Business Structure',
 'Premise Address',
 'Status',
 'Issue Date',
 'Expiration Date',
 'Activities',
 'Adult-Use/Medicinal']

In [5]:
#Note that there are multiple delimiters: a colon (":"), a dash ("-"), a comma (","), and a blank space (" ")
california_data_df["Business Contact Information"].head()

0    PDC, LLC : Email- license@pmcoc.com : Phone- 7...
1    WE CARE CAT CITY INC.  : GUIDE : Email- George...
2    GREENFIELD HERBS  : Greenfield Herbs : Email- ...
3    BEAVER FREIGHT, INC. : Email- mcastaneda85@gma...
4    SESPE CREEK COLLECTIVE, INC. : Email- info@ses...
Name: Business Contact Information, dtype: object

In [6]:
##Extract and separate "Business Name" from the california_data_df["Business Contact Information"] column

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new dataframe with split value columns 
new = california_data_df["Business Contact Information"].str.split(":", n = 1, expand = True) 
  
# making separate "Business Name" column from new data frame 
california_data_df["Business Name"]= new[0] 
  
# making separate "Contact Information" column from new data frame 
california_data_df["Contact Information"]= new[1] 

# Dropping old "Business Contact Information" column
california_data_df.drop(columns =["Business Contact Information"], inplace = True) 

#california_data_df display with the new columns
## Note: california_data_df["Business Name"] and california_data_df["Contact Information"] BOTH need cleaning 
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Business Name,Contact Information
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",Email- license@pmcoc.com : Phone- 7144857379 ...
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,WE CARE CAT CITY INC.,GUIDE : Email- Georgedggroup@gmail.com : Phon...
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,GREENFIELD HERBS,Greenfield Herbs : Email- carol@greenfieldher...
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal,"BEAVER FREIGHT, INC.",Email- mcastaneda85@gmail.com : Phone- 909963...
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",Email- info@sespe.org : Phone- 8557229333 : W...


In [7]:
##Extract the occasional extraneous "Business Name" info from the california_data_df["Contact Information"] column 

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Contact Information"].str.split("Email-", n = 1, expand = True) 
  
# making separate "Extra Business Name Information" column from new data frame that contains the occasional extraneous "Business Name" info.
california_data_df["Extra Business Name Information"]= new[0] 
  
# making separate "Contact Information2"column from new data frame 
california_data_df["Contact Information2"]= new[1] 

# Dropping old "Contact Information" column
california_data_df.drop(columns =["Contact Information"], inplace = True) 

#california_data_df display with the new columns
## Note: we must now combine california_data_df["Business Name"] with california_data_df["Extra Business Name Information"]
## Note: california_data_df["Contact Information2"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Business Name,Extra Business Name Information,Contact Information2
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",,license@pmcoc.com : Phone- 7144857379 : Websi...
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,WE CARE CAT CITY INC.,GUIDE :,Georgedggroup@gmail.com : Phone- 7606737220
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,GREENFIELD HERBS,Greenfield Herbs :,carol@greenfieldherbs.com : Phone- 8318095469
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal,"BEAVER FREIGHT, INC.",,mcastaneda85@gmail.com : Phone- 9099638849
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",,info@sespe.org : Phone- 8557229333 : Website-...


In [8]:
#Combine california_data_df["Business Name"] with california_data_df["Extra Business Name Information"] and clean
california_data_df['Company Name'] = california_data_df['Business Name'].str.cat(california_data_df['Extra Business Name Information'],sep=" ")
california_data_df["Company Name"] = california_data_df["Company Name"].str.replace(':,?' , '')

# Dropping california_data_df["Business Name]" and california_data_df["Extra Business Name Information"] columns
california_data_df.drop(columns =["Business Name"], inplace = True)
california_data_df.drop(columns =["Extra Business Name Information"], inplace = True)

#california_data_df display with the new column (california_data_df["Company Name"])
##Note: california_data_df["Contact Information2"] still needs cleaning
california_data_df.head()


,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Contact Information2,Company Name
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,license@pmcoc.com : Phone- 7144857379 : Websi...,"PDC, LLC"
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,Georgedggroup@gmail.com : Phone- 7606737220,WE CARE CAT CITY INC. GUIDE
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,carol@greenfieldherbs.com : Phone- 8318095469,GREENFIELD HERBS Greenfield Herbs
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal,mcastaneda85@gmail.com : Phone- 9099638849,"BEAVER FREIGHT, INC."
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,info@sespe.org : Phone- 8557229333 : Website-...,"SESPE CREEK COLLECTIVE, INC."


In [9]:
##Extract and separate "Email" from the california_data_df["Contact Information2"] column

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Contact Information2"].str.split(":", n = 1, expand = True) 
  
# making separate "Business Name" column from new data frame 
california_data_df["Email"]= new[0] 
  
# making separate "Contact Information" column from new data frame 
california_data_df["Contact Information3"]= new[1] 

# Dropping california_data_df["Contact Information2"] column
california_data_df.drop(columns =["Contact Information2"], inplace = True) 

#california_data_df display with the new columns
##Note: california_data_df["Contact Information3"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Contact Information3
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,Phone- 7144857379 : Website- www.herodistro.com
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,WE CARE CAT CITY INC. GUIDE,Georgedggroup@gmail.com,Phone- 7606737220
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,GREENFIELD HERBS Greenfield Herbs,carol@greenfieldherbs.com,Phone- 8318095469
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal,"BEAVER FREIGHT, INC.",mcastaneda85@gmail.com,Phone- 9099638849
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,Phone- 8557229333 : Website- www.sespe.org


In [10]:
##Extract and separate "Phone Number" from the california_data_df["Contact Information3"] column.

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Contact Information3"].str.split(":", n = 1, expand = True) 
  
# making separate "Business Name" column from new data frame 
california_data_df["Phone Number"]= new[0] 
  
# making separate "Contact Information" column from new data frame 
california_data_df["Contact Information4"]= new[1] 

# Dropping california_data_df["Contact Information"] column
california_data_df.drop(columns =["Contact Information3"], inplace = True) 

#california_data_df display with the new columns
##Note: california_data_df["Phone Number"] needs to contain only the digits of the phone number
##Note: california_data_df["Contact Information4"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Phone Number,Contact Information4
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,Phone- 7144857379,Website- www.herodistro.com
1,C11-0000035-LIC,Cannabis - Distributor License,Alicia Schultz,Corporation,"Cathedral City, CA 92234 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,WE CARE CAT CITY INC. GUIDE,Georgedggroup@gmail.com,Phone- 7606737220,None
2,C11-0000036-LIC,Cannabis - Distributor License,Connie Bauer,Corporation,"KING CITY, CA 93930 County: MONTEREY",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,GREENFIELD HERBS Greenfield Herbs,carol@greenfieldherbs.com,Phone- 8318095469,None
3,C11-0000037-LIC,Cannabis - Distributor License,Marcella Castaneda,Corporation,"ADELANTO, CA 92301 County: SAN BERNARDINO",Active,05/02/2019,05/01/2020,N/A for this license type,Medicinal,"BEAVER FREIGHT, INC.",mcastaneda85@gmail.com,Phone- 9099638849,None
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,Phone- 8557229333,Website- www.sespe.org


In [11]:
#Clean up california_data_df["Phone Number"] so that it shows only the digits of phone number
#(ie. Remove the string ("Phone-") from the column

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Phone Number"].str.split("-", n = 1, expand = True) 
  
# making separate "Phone str" column from new data frame to extract the unwanted string
california_data_df["Phone str"]= new[0] 
  
# making separate "Telephone Number" column from new data frame 
california_data_df["Telephone Number"]= new[1] 

# Dropping california_data_df["Phone str"] and california_data_df["Phone Number"] columns
california_data_df.drop(columns =["Phone Number"], inplace = True)
california_data_df.drop(columns =["Phone str"], inplace = True) 

#california_data_df display with the new columns
##Note: california_data_df["Contact Information4"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Contact Information4,Telephone Number
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,Website- www.herodistro.com,7144857379
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,Website- www.sespe.org,8557229333
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,"400 E. Sunny Dunes RD Palm Springs, CA 92264 C...",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,Website- psaorganica.com,7602506477
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,"2675 VENTURA RD PORT HUENEME, CA 930412054 Cou...",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,Website- fteusa.com,8054141394
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,"OAKLAND, CA 946214432 County: ALAMEDA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,Website- www.codasignature.com,8444202632


In [12]:
#Clean up the california_data_df["Contact Information4"] column so that it shows only the actual website address
#(ie. Remove the string ("Website-") from the column

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Contact Information4"].str.split("-", n = 1, expand = True) 
  
# making separate "Website str" column from new data frame to extract the unwanted string
california_data_df["Website str"]= new[0]     
  
# making separate "Website Address" column from new data frame 
california_data_df["Website Address"]= new[1] 

# Dropping california_data_df["Website str"] and california_data_df["Contact Information4"] columns
california_data_df.drop(columns =["Contact Information4"], inplace = True)
california_data_df.drop(columns =["Website str"], inplace = True) 

#california_data_df display with the new columns
california_data_df.head()

##SECTION 2.1 Completed
##################################################################################################################

,License Number,License Type,Business Owner,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,"CATHEDRAL CITY, CA 922347624 County: RIVERSIDE",Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,"408 BRYANT CIR OJAI, CA 93023 County: VENTURA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,"400 E. Sunny Dunes RD Palm Springs, CA 92264 C...",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,"2675 VENTURA RD PORT HUENEME, CA 930412054 Cou...",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,"OAKLAND, CA 946214432 County: ALAMEDA",Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com


In [13]:
#Business Contact Information column cleanup:
california_data_df["Premise Address"].head()
#Note that there are multiple delimiters: a colon (":"), a comma (","), and a blank space (" ")
#Note that the zip codes have either 5 or 9 digits

0        CATHEDRAL CITY, CA 922347624 County: RIVERSIDE
4         408 BRYANT CIR OJAI, CA 93023 County: VENTURA
7     400 E. Sunny Dunes RD Palm Springs, CA 92264 C...
8     2675 VENTURA RD PORT HUENEME, CA 930412054 Cou...
13                OAKLAND, CA 946214432 County: ALAMEDA
Name: Premise Address, dtype: object

In [14]:
##Extract and separate "County" from the california_data_df["Premise Address"] column

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Premise Address"].str.split(":", n = 1, expand = True) 
  
# making separate "Business Name" column from new data frame 
california_data_df["Premise Address2"]= new[0] 
  
# making separate "Contact Information" column from new data frame 
california_data_df["County"]= new[1] 

# Dropping california_data_df["Premise Address"] column
california_data_df.drop(columns =["Premise Address"], inplace = True) 

#california_data_df display with the new columns
##Note: california_data_df["County"] still needs cleaning
##Note: california_data_df["Premise Address2"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,Premise Address2,County
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,"CATHEDRAL CITY, CA 922347624 County",RIVERSIDE
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,"408 BRYANT CIR OJAI, CA 93023 County",VENTURA
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,"400 E. Sunny Dunes RD Palm Springs, CA 92264 C...",RIVERSIDE
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,"2675 VENTURA RD PORT HUENEME, CA 930412054 County",VENTURA
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,"OAKLAND, CA 946214432 County",ALAMEDA


In [15]:
#Clean up california_data_df["County"] -- Problem: all letters in the column are capitalized, and we need to fix this

#Adjust the case structure so that only the first letter in "County" is capitalized while all others are lower case
california_data_df["County"] = california_data_df["County"].str.title()

#california_data_df display with the new columns
##Note: california_data_df["Premise Address3"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,Premise Address2,County
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,"CATHEDRAL CITY, CA 922347624 County",Riverside
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,"408 BRYANT CIR OJAI, CA 93023 County",Ventura
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,"400 E. Sunny Dunes RD Palm Springs, CA 92264 C...",Riverside
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,"2675 VENTURA RD PORT HUENEME, CA 930412054 County",Ventura
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,"OAKLAND, CA 946214432 County",Alameda


In [16]:
#Clean up california_data_df["Premise Address2"] so that the superfluous string "County" can be excised
#(ie. Remove the string ("County") now that the actual county has been extracted into its own column

#Drop the 'County' string from the "Premise Address2" columnn
california_data_df["Premise Address3"] = california_data_df["Premise Address2"].str.replace('County,?' , '')

# Dropping old "Premise Address2" column
california_data_df.drop(columns =["Premise Address2"], inplace = True)

#california_data_df display with the new columns
##Note: california_data_df["Premise Address3"] still needs cleaning
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,County,Premise Address3
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,Riverside,"CATHEDRAL CITY, CA 922347624"
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,Ventura,"408 BRYANT CIR OJAI, CA 93023"
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,Riverside,"400 E. Sunny Dunes RD Palm Springs, CA 92264"
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,Ventura,"2675 VENTURA RD PORT HUENEME, CA 930412054"
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,Alameda,"OAKLAND, CA 946214432"


In [17]:
##Extract and separate "Address" from the "Premise Address3" column.

# dropping null value columns to avoid errors 
california_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = california_data_df["Premise Address3"].str.split(",", n = 1, expand = True) 
  
# making separate "Address" column from new data frame 
california_data_df["Address_misc"]= new[0] 
  
# making separate "State/Zip Code" column from new data frame 
california_data_df["State/Zip Code"]= new[1] 

# Dropping old "Premise Address3" column
california_data_df.drop(columns =["Premise Address3"], inplace = True) 

#california_data_df display with the new columns
## Note: california_data_df["Address misc"] and california_data_df["StateSip Code"] BOTH still need cleaning 
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,County,Address_misc,State/Zip Code
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,Riverside,CATHEDRAL CITY,CA 922347624
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,Ventura,408 BRYANT CIR OJAI,CA 93023
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,Riverside,400 E. Sunny Dunes RD Palm Springs,CA 92264
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,Ventura,2675 VENTURA RD PORT HUENEME,CA 930412054
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,Alameda,OAKLAND,CA 946214432


In [18]:
#Drop the 'CA' string from the State/Zip Code column, since the State information is superfluous
california_data_df["Zip Code"] = california_data_df["State/Zip Code"].str.replace('CA,?' ,'')

# Dropping old "State/Zip Code" column
california_data_df.drop(columns =["State/Zip Code"], inplace = True)

#california_data_df display with the new columns
## Note: california_data_df["Address_misc"] and california_data_df["Zip Code"] BOTH still need cleaning 
california_data_df.head()

,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,County,Address_misc,Zip Code
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,Riverside,CATHEDRAL CITY,922347624
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,Ventura,408 BRYANT CIR OJAI,93023
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,Riverside,400 E. Sunny Dunes RD Palm Springs,92264
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,Ventura,2675 VENTURA RD PORT HUENEME,930412054
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,Alameda,OAKLAND,946214432


In [19]:
#Note: Some of the data in the "Zip Code" column has 9 digits, wihle others have 5 digits
california_data_df["Zip Code"].head()
#Need to clean up the "Zip Code" data so that the zip code is the standard 5-digit code

0       922347624 
4           93023 
7           92264 
8       930412054 
13      946214432 
Name: Zip Code, dtype: object

In [20]:
#Clean up "Zip Code" column so that the zip code is the standard 5-digit code, and not the 9-digit code that appers sporadically above
california_data_df['Zip Code'] = california_data_df['Zip Code'].str[:7]
california_data_df.head()


,License Number,License Type,Business Owner,Business Structure,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,Company Name,Email,Telephone Number,Website Address,County,Address_misc,Zip Code
0,C11-0000034-LIC,Cannabis - Distributor License,Bernard Steimann,Limited Liability Company,Active,05/02/2019,05/01/2020,N/A for this license type,BOTH,"PDC, LLC",license@pmcoc.com,7144857379,www.herodistro.com,Riverside,CATHEDRAL CITY,92234
4,C10-0000028-LIC,Cannabis - Retailer License,Chelsea Sutula: Chelsea Sutula,Corporation,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"SESPE CREEK COLLECTIVE, INC.",info@sespe.org,8557229333,www.sespe.org,Ventura,408 BRYANT CIR OJAI,93023
7,C10-0000029-LIC,Cannabis - Retailer License,Julie Montante,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,PALM SPRINGS ORGANICS LLC,jmexotic@aol.com,7602506477,psaorganica.com,Riverside,400 E. Sunny Dunes RD Palm Springs,92264
8,C10-0000030-LIC,Cannabis - Retailer License,David Moss,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"DBO INVESTMENTS PH, LLC From The Earth",d@fteusa.com,8054141394,fteusa.com,Ventura,2675 VENTURA RD PORT HUENEME,93041
13,C11-0000027-LIC,Cannabis - Distributor License,Elizabeth Cooke,Limited Liability Company,Active,05/01/2019,04/30/2020,N/A for this license type,BOTH,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",elizabeth@codasignature.com,8444202632,www.codasignature.com,Alameda,OAKLAND,94621


In [21]:
#Choose the most important columns for the next part of the ETL Project
california_data_df = california_data_df[["Company Name","Website Address","County","Zip Code"]]
#Rename column names so that they are SQL feiendly
california_data_df.columns=["Company_Name","Website_Address","County","Zip_Code"]
california_data_df.head()
california_data_df.reset_index(drop = True)

,Company_Name,Website_Address,County,Zip_Code
0,"PDC, LLC",www.herodistro.com,Riverside,92234
1,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023
2,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264
3,"DBO INVESTMENTS PH, LLC From The Earth",fteusa.com,Ventura,93041
4,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",www.codasignature.com,Alameda,94621
5,"REVIVE ORGANICS, LLC Revive Pure Life",www.revivepurelife.com,Nevada,95959
6,CI DISTRIBUTION LLC Claybourne Distribution,www.claybournedistribution.com,Riverside,92571
7,BLUM SAN LEANDRO,www.letsblum.com,Alameda,94577
8,CROSSTOWN CALIFORNIA HOLDINGS LLC N-FUZED; ...,https://www.nfuzed.com/,San Luis Obispo,93433
9,SUNSHINE HOLISTIC Sovereign,www.sovereign707.org,Mendocino,95437


In [22]:
#lets load the Latitude and Longitude coordinates from the csv we created from the API
lat_lng= pd.read_csv("../ETL_project/lat_lng.csv")
lat_lng.columns=["A","Latitude", "Longitude"]
lat_lng.reset_index(drop=True)
lat_lng = lat_lng.reset_index(drop=True)

lat_lng.head()

,A,Latitude,Longitude
0,0,33.780539,-116.466804
1,1,34.443072,-119.237926
2,2,33.812404,-116.543471
3,3,34.178267,-119.196389
4,4,37.804364,-122.271114


In [23]:
#Merge the Latitude/Longitude data in with california_data_df
california_data_df = pd.merge(california_data_df, lat_lng, left_index=True, right_index=True)
#california_data_df.drop(["A"], axis=1, inplace=True)
california_data_df.head()

,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude
0,"PDC, LLC",www.herodistro.com,Riverside,92234,0,33.780539,-116.466804
4,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023,4,37.804364,-122.271114
7,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264,7,37.703576,-122.168828
8,"DBO INVESTMENTS PH, LLC From The Earth",fteusa.com,Ventura,93041,8,35.121642,-120.621282
13,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",www.codasignature.com,Alameda,94621,13,33.770050,-118.193740


In [24]:
## Source = https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

In [25]:
#California_Census_Data
census_data = "../ETL_project/california_2016_census_data.csv"
census_data_df = pd.read_csv(census_data, encoding="utf-8")
census_data_df.head(12)


/Users/holbrook/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,CALIFORNIA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144
0,Individual Income Tax Returns: \nSelected Inco...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Money amounts are in thousands of dollars],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZIP\ncode [1],Size of adjusted gross income,Number of returns,Number of single returns,Number of joint returns,Number of head of household returns,Number with paid preparer's signature,Number of exemptions,Number of dependents,Number of volunteer prepared returns [2],...,Total tax liability [10],NaN,Total additional Medicare tax,NaN,Net investment income tax,NaN,Tax due at time of filing [11],NaN,Overpayments refunded [12],NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Total,...,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount,Number of returns,Amount
4,NaN,NaN,(1),(2),(3),(4),(5),(6),(7),(8),...,(134),(135),(136),(137),(138),(139),(140),(141),(142),(143)
5,00000,Total,"17,532,400","8,548,270","6,211,790","2,523,550","10,861,110","35,441,880","12,566,880","292,860",...,"13,713,370","224,205,553","649,400","1,412,144","661,770","3,184,569","4,066,410","24,310,929","12,583,510","41,461,583"
6,00000,"$1 under $25,000",5975570,4192510,774400,940840,3507450,8877550,2969880,182750,...,3119640,3147718,20,9,70,110,843690,783942,4707030,9411130
7,00000,"$25,000 under $50,000",4168180,2113080,1075050,912660,2571270,8672900,3453670,85770,...,3373610,9189547,20,5,60,23,782510,1423468,3309570,8311140
8,00000,"$50,000 under $75,000",2328820,1019940,919620,344820,1454740,5086710,1842140,20100,...,2203430,12647108,60,14,60,27,614100,1616862,1657290,4443591
9,00000,"$75,000 under $100,000",1496930,521570,794530,154380,952050,3447440,1155340,3590,...,1470470,13647614,130,32,40,14,429980,1449379,1016910,3367963


In [26]:
#Find the pertinent data and their columns and rename the columns
census_data_df.rename(columns={"CALIFORNIA":"Zip Code"}, inplace=True)
census_data_df.rename(columns={"Unnamed: 1":"Income Bracket"}, inplace=True)
census_data_df.rename(columns={"Unnamed: 65":"Number of Tax Returns"}, inplace=True)
census_data_df.rename(columns={"Unnamed: 66":"Total Income"}, inplace=True)

list(census_data_df)

['Zip Code',
 'Income Bracket',
 'Unnamed: 2',
 'Unnamed: 3',
 'Unnamed: 4',
 'Unnamed: 5',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9',
 'Unnamed: 10',
 'Unnamed: 11',
 'Unnamed: 12',
 'Unnamed: 13',
 'Unnamed: 14',
 'Unnamed: 15',
 'Unnamed: 16',
 'Unnamed: 17',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Unnamed: 22',
 'Unnamed: 23',
 'Unnamed: 24',
 'Unnamed: 25',
 'Unnamed: 26',
 'Unnamed: 27',
 'Unnamed: 28',
 'Unnamed: 29',
 'Unnamed: 30',
 'Unnamed: 31',
 'Unnamed: 32',
 'Unnamed: 33',
 'Unnamed: 34',
 'Unnamed: 35',
 'Unnamed: 36',
 'Unnamed: 37',
 'Unnamed: 38',
 'Unnamed: 39',
 'Unnamed: 40',
 'Unnamed: 41',
 'Unnamed: 42',
 'Unnamed: 43',
 'Unnamed: 44',
 'Unnamed: 45',
 'Unnamed: 46',
 'Unnamed: 47',
 'Unnamed: 48',
 'Unnamed: 49',
 'Unnamed: 50',
 'Unnamed: 51',
 'Unnamed: 52',
 'Unnamed: 53',
 'Unnamed: 54',
 'Unnamed: 55',
 'Unnamed: 56',
 'Unnamed: 57',
 'Unnamed: 58',
 'Unnamed: 59',
 'Unnamed: 60',
 'Unnamed: 61',
 'Unnamed: 62',


In [27]:
#Choose the most pertinent columns for the census part of the ETL Project
census_data_df = census_data_df[["Zip Code","Income Bracket","Number of Tax Returns","Total Income"]]
census_data_df.head(19)

,Zip Code,Income Bracket,Number of Tax Returns,Total Income
0,Individual Income Tax Returns: \nSelected Inco...,NaN,NaN,NaN
1,[Money amounts are in thousands of dollars],NaN,NaN,NaN
2,ZIP\ncode [1],Size of adjusted gross income,State and local income taxes,NaN
3,NaN,NaN,Number of returns,Amount
4,NaN,NaN,(64),(65)
5,00000,Total,"5,112,640","80,160,634"
6,00000,"$1 under $25,000",116550,229306
7,00000,"$25,000 under $50,000",457690,815490
8,00000,"$50,000 under $75,000",721460,2138961
9,00000,"$75,000 under $100,000",807510,3699871


In [28]:
#Read in new California_Census_Data
census_data2 = "../ETL_project/census_clean.csv"
census_data2_df = pd.read_csv(census_data2)
#Rename the column names
census_data2_df.columns = ["Zip Code","Total Income","Number of Tax Returns"]
census_data2_df.head()

,Zip Code,Total Income,Number of Tax Returns
0,90001,619635000,21670
1,90001,171277000,11890
2,90001,245583000,7020
3,90001,116362000,1940
4,90001,47795000,560


In [29]:
#Aggregate the group data per Zip Code via groupby function
aggregate_census_data_df = (census_data2_df.groupby('Zip Code').sum()).reset_index()
aggregate_census_data_df.head()

,Zip Code,Total Income,Number of Tax Returns
0,90001,1239270000,43340
1,90002,1126344000,39780
2,90003,1461620000,54580
3,90004,3991172000,55620
4,90005,1615486000,31700


In [30]:
#Create a new "Zip Code Income" column
aggregate_census_data_df["Zip Code Income"] = aggregate_census_data_df["Total Income"]/aggregate_census_data_df["Number of Tax Returns"]
aggregate_census_data_df["Zip Code Income"] = aggregate_census_data_df["Zip Code Income"].round()
aggregate_census_data_df.head()

,Zip Code,Total Income,Number of Tax Returns,Zip Code Income
0,90001,1239270000,43340,28594.0
1,90002,1126344000,39780,28314.0
2,90003,1461620000,54580,26779.0
3,90004,3991172000,55620,71758.0
4,90005,1615486000,31700,50962.0


In [31]:
#Reformat the "Zip Code Income" column so that it includes comma delimiters per thousand
#This is aesthetically more pleasing on the final product
aggregate_census_data_df["Zip Code Income"] = aggregate_census_data_df["Zip Code Income"].apply("{:,}".format)
aggregate_census_data_df.head()

,Zip Code,Total Income,Number of Tax Returns,Zip Code Income
0,90001,1239270000,43340,"28,594.0"
1,90002,1126344000,39780,"28,314.0"
2,90003,1461620000,54580,"26,779.0"
3,90004,3991172000,55620,"71,758.0"
4,90005,1615486000,31700,"50,962.0"


In [32]:
##Clean the "Zip Code Income" column so that the ".0" end of the string is eliminated
# dropping null value columns to avoid errors 
aggregate_census_data_df.dropna(inplace = True) 
  
# new data frame with split value columns 
new = aggregate_census_data_df["Zip Code Income"].str.split(".", n = 1, expand = True) 
  
# making separate "Business Name" column from new data frame 
aggregate_census_data_df["Per Capita Income"]= new[0] 
  
# making separate "misc" column from new data frame 
aggregate_census_data_df["misc"]=new[1]

# Dropping aggregate_census_data["misc"] column
# Dropping aggregate_census_data["Zip Code Income"] column
# Dropping aggregate_census_data["Total Income"] column
# Dropping aggregate_census_data["Number of Tax Returns"] column
aggregate_census_data_df.drop(columns =["misc"], inplace = True) 
aggregate_census_data_df.drop(columns =["Zip Code Income"], inplace = True) 
aggregate_census_data_df.drop(columns =["Total Income"], inplace = True) 
aggregate_census_data_df.drop(columns =["Number of Tax Returns"], inplace = True) 

In [33]:
#Rename columns so that they are SQL friendly
aggregate_census_data_df.columns=["Zip_Code","Per_Capita_Income"]
#display with the new columns
aggregate_census_data_df.head()

,Zip_Code,Per_Capita_Income
0,90001,"28,594"
1,90002,"28,314"
2,90003,"26,779"
3,90004,"71,758"
4,90005,"50,962"


In [34]:
list(aggregate_census_data_df)

['Zip_Code', 'Per_Capita_Income']

In [35]:
#CREATE DATABASE california_data_db;

# USE california_data_db;

# CREATE TABLE california_data(
# 	id INT PRIMARY KEY,
#     Company_Name TEXT,
#     Website_Address TEXT,
#     COUNTY TEXT,
#     Zip_Code TEXT
#     A INT
#     Latitude TEXT
#     Longitude TEXT
# );

# CREATE TABLE census_data(
# 	id INT PRIMARY KEY,
#     Zip_Code TEXT,
#     Per_Capita_Income TEXT
# );


In [84]:
connection_string = "sqlite:///CALIFORNIA_ETL_data_db.sqlite"
engine = create_engine(connection_string)

In [85]:
engine.table_names()

['california_data', 'census_data']

In [57]:
engine.execute("CREATE TABLE california_data(id INT PRIMARY KEY, Company_Name TEXT, Website_Address TEXT, County TEXT, Zip_Code INT, A INT, Latitude INT, Longitude INT);")


In [58]:
engine.execute("CREATE TABLE census_data(id INT PRIMARY KEY, Zip_Code TEXT, Per_Capita_Income TEXT);")


In [86]:
california_data_df.to_sql(name="california_data", con=engine, if_exists="append", index=False)


In [83]:
california_data_df["Zip_Code"]=california_data_df["Zip_Code"].astype("int64")
california_data_df.dtypes

Company_Name        object
Website_Address     object
County              object
Zip_Code             int64
A                    int64
Latitude           float64
Longitude          float64
dtype: object

In [60]:
pd.read_sql_query("SELECT * FROM california_data",con=engine).head()


,id,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude
0,None,"PDC, LLC",www.herodistro.com,Riverside,92234,0,33.780539,-116.466804
1,None,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023,4,37.804364,-122.271114
2,None,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264,7,37.703576,-122.168828
3,None,"DBO INVESTMENTS PH, LLC From The Earth",fteusa.com,Ventura,93041,8,35.121642,-120.621282
4,None,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",www.codasignature.com,Alameda,94621,13,33.770050,-118.193740


In [61]:
aggregate_census_data_df.to_sql(name = "census_data", con=engine, if_exists="append", index=False)


In [79]:
aggregate_census_data_df.dtypes

Zip_Code              int64
Per_Capita_Income    object
dtype: object

In [62]:
pd.read_sql_query("SELECT * FROM census_data",con=engine).head()


,id,Zip_Code,Per_Capita_Income
0,None,90001,"28,594"
1,None,90002,"28,314"
2,None,90003,"26,779"
3,None,90004,"71,758"
4,None,90005,"50,962"


In [ ]:
## Why are we getting "none" for both id columns ??? --- solved

In [107]:
query = """
SELECT
*
FROM california_data A
INNER JOIN census_data B on A.Zip_Code = B.Zip_Code GROUP BY A.Zip_Code
"""
pd.read_sql_query(query, con=engine).head()

,id,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude,id,Zip_Code,Per_Capita_Income
0,None,Indus Holding Company Cypress Manufacturing...,20 Quail Run Circle,Los Angeles,90007,953,33.713353,-117.873104,None,90007,"30,305"
1,None,"COLLECTIVE PHARM, INC.",www.highnotewest.com,Los Angeles,90016,635,33.780539,-116.466804,None,90016,"42,658"
2,None,"MELROSE HERBAL COLLECTIVE, INC. LA Cannabis...",lacannabisco.com,Los Angeles,90019,879,37.804364,-122.271114,None,90019,"53,707"
3,None,Spliffin Group,spliffin.com,Los Angeles,90021,440,32.769641,-117.132772,None,90021,"105,012"
4,None,TEDS BUDZ,"11430 Burbank blvd, 217",Los Angeles,90023,591,37.973535,-122.531087,None,90023,"31,063"


In [ ]:
## question: what is best way to make this join work, since the "id" columna are not working as planned ??


In [ ]:
############################################################


In [92]:
california_data_df.head()

,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude
0,"PDC, LLC",www.herodistro.com,Riverside,92234,0,33.780539,-116.466804
4,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023,4,37.804364,-122.271114
7,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264,7,37.703576,-122.168828
8,"DBO INVESTMENTS PH, LLC From The Earth",fteusa.com,Ventura,93041,8,35.121642,-120.621282
13,"THE GROW FOUNDRY CALIFORNIA, LLC Coda Signa...",www.codasignature.com,Alameda,94621,13,33.770050,-118.193740


In [93]:
aggregate_census_data_df.head()

,Zip_Code,Per_Capita_Income
0,90001,"28,594"
1,90002,"28,314"
2,90003,"26,779"
3,90004,"71,758"
4,90005,"50,962"


In [94]:
merged_data_df = pd.merge(california_data_df,aggregate_census_data_df,on="Zip_Code")


In [112]:
merged_data_df.head()


,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude,Per_Capita_Income
0,"PDC, LLC",www.herodistro.com,Riverside,92234,0,33.780539,-116.466804,"43,369"
1,"MIRACLE EDUCATION DISTRIBUTORS, INC Miracle...",1717 E. Vista Chino A7 Unit 824,Riverside,92234,376,39.219061,-121.061061,"43,369"
2,CC PROS,www.catcitypros.com,Riverside,92234,961,34.567991,-117.400136,"43,369"
3,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023,4,37.804364,-122.271114,"87,372"
4,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264,7,37.703576,-122.168828,"77,858"


In [113]:

merged_data_df.head()


,Company_Name,Website_Address,County,Zip_Code,A,Latitude,Longitude,Per_Capita_Income,Count
0,"PDC, LLC",www.herodistro.com,Riverside,92234,0,33.780539,-116.466804,"43,369",NaN
1,"MIRACLE EDUCATION DISTRIBUTORS, INC Miracle...",1717 E. Vista Chino A7 Unit 824,Riverside,92234,376,39.219061,-121.061061,"43,369",NaN
2,CC PROS,www.catcitypros.com,Riverside,92234,961,34.567991,-117.400136,"43,369",NaN
3,"SESPE CREEK COLLECTIVE, INC.",www.sespe.org,Ventura,93023,4,37.804364,-122.271114,"87,372",NaN
4,PALM SPRINGS ORGANICS LLC,psaorganica.com,Riverside,92264,7,37.703576,-122.168828,"77,858",NaN
